## Notebook 5 - Further Anlalysis of RCT output.

This notebook shows a few other ways RCT output can be visualised. If you have any ideas of your own, the end of this notebook would be a good place to add it. 

In [ ]:
# Suppress depreciation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Import the required modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import helper_functions

### Read segment level data from forest treefile generated using treeinfo

In [ ]:
tree_segments_df = helper_functions.treeinfo_attributes_segment('../data/Mak1_clipped_raycloud_inside_trees_info.txt')
tree_segments_df

### Get the plot total volume

In [ ]:
plot_volume = tree_segments_df['volume'].sum() 
print('The total volume of the plot is:', plot_volume, 'm^3')

### Plot the mean segment volume by height with bins of 1 meter for the whole plot

In [ ]:
# Normalize 'z' values to start from 0
tree_segments_df.loc[:, 'z_normalized'] = tree_segments_df['z'] - tree_segments_df['z'].min()

# Round the normalized 'z' values to the nearest meter
tree_segments_df.loc[:, 'z_rounded'] = np.round(tree_segments_df['z_normalized'])

# Group by the rounded normalized height ('z_rounded') and calculate the mean volume for each bin
mean_volume_by_height = tree_segments_df.groupby('z_rounded')['volume'].mean()

# Plot the mean volume by height with bins of 1 meter
plt.figure(figsize=(10, 8))
plt.plot(mean_volume_by_height.values, mean_volume_by_height.index, linestyle='-')

# Add labels and title
plt.ylabel('Height (m)')
plt.xlabel('Mean Segment Volume [m^3]')
plt.title('Mean Segment Volume by Height (1m bins)')

# Show the plot
plt.grid(True)
plt.show()


### Plot the mean segment volume by height with bins of 1 meter for a single tree

In [ ]:
# Select a single tree from the segment dataframe
tree_segments_df_single = tree_segments_df.loc[tree_segments_df['tree_id'] == 22]

# Normalize 'z' values to start from 0
tree_segments_df_single.loc[:, 'z_normalized'] = tree_segments_df_single['z'] - tree_segments_df_single['z'].min()

# Round the normalized 'z' values to the nearest meter
tree_segments_df_single.loc[:, 'z_rounded'] = np.round(tree_segments_df_single['z_normalized'])

# Group by the rounded normalized height ('z_rounded') and calculate the mean volume for each bin
mean_volume_by_height = tree_segments_df_single.groupby('z_rounded')['radius'].mean()

# Plot the mean volume by height with bins of 1 meter
plt.figure(figsize=(10, 8))
plt.plot(mean_volume_by_height.values, mean_volume_by_height.index, linestyle='-')

# Add labels and title
plt.ylabel('Height (m)')
plt.xlabel('Mean Segment Volume [m^3]')
plt.title('Mean Segment Volume by Height (1m bins)')

# Show the plot
plt.grid(True)
plt.show()


### Read in the tree level data prepared in previous notebooks

In [ ]:
rct_df = pd.read_csv('../data/rct_translated_joined_data.csv')
rct_df

### Plot Height by DBH for trees with DBH over 10cm

In [ ]:
# Get DBH over 10cm
rct_df_filtered = rct_df[rct_df['DBH'] > 0.1]

# Plot the DBH vs Height
plt.figure(figsize=(10, 10))
plt.scatter(rct_df_filtered['DBH'], rct_df_filtered['height'])
plt.xlabel('DBH')
plt.ylabel('Height')
plt.title('DBH vs Height (DBH > 10cm)')
plt.grid(True)
plt.show()


### Plot tree locations with stem basal area

In [ ]:
plt.figure(figsize=(10, 8))

rct_df_filtered['stem_area'] = np.pi * (rct_df_filtered['DBH'] ** 2) * 100
scatter = plt.scatter(rct_df_filtered['x'], rct_df_filtered['y'], s=rct_df_filtered['stem_area'])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Tree Locations with Stem Basal Area')
plt.grid(True)

# This is text, adjust the placement and text as needed
plt.text(0.99, 0.01, 'Note: Stem surface area is scaled by a factor of 100 from the actual area',
         verticalalignment='bottom', horizontalalignment='right',
         transform=plt.gca().transAxes,  # This aligns the text to the axes
         color='blue', fontsize=8)

plt.show()

### Plotting tree locations with stem basal area and crown radius

In [ ]:
import matplotlib.colors as mcolors

# Setup the figure and axes
fig, ax = plt.subplots(figsize=(10, 8))

# Define the colormap for the crown radius circles
cmap = plt.cm.viridis

# Normalize the height column for the colormap to color the crown radius circles by tree height
norm = mcolors.Normalize(vmin=rct_df_filtered['height'].min(), vmax=rct_df_filtered['height'].max())

# Adding circles to represent crown radius, colored by height
for idx, row in rct_df_filtered.iterrows():
    color = cmap(norm(row['height']))  # Determine circle color based on tree height
    crown_circle = plt.Circle((row['x_translated'], row['y_translated']), row['crown_radius'], color=color, fill=True, alpha=0.4, linewidth=1.5)
    ax.add_artist(crown_circle)

# Plotting tree locations with stem area as black circles
ax.scatter(rct_df_filtered['x_translated'], rct_df_filtered['y_translated'], s=rct_df_filtered['stem_area'], c='black', label='Stem Area', alpha=0.6, edgecolors='none')

# Set x and y limits
ax.set_xlim(rct_df_filtered['x_translated'].min() - 10, rct_df_filtered['x_translated'].max() + 10)
ax.set_ylim(rct_df_filtered['y_translated'].min() - 10, rct_df_filtered['y_translated'].max() + 10)

# Customizing the plot
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Tree Locations with Stem Area and Crown Radius by Height')
ax.grid(True)

# Adding a colorbar to indicate the scale for tree height
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, ax=ax, label='Height')

# This is text, adjust the placement and text as needed
plt.text(0.99, 0.01, 'Note: Stem surface area is scaled by a factor of 100 from the actual area',
         verticalalignment='bottom', horizontalalignment='right',
         transform=plt.gca().transAxes,  # This aligns the text to the axes
         color='blue', fontsize=8)

plt.show()